# *Universidad Tecnológica de Bolívar*
## *Maestría en Ingeniería*  
### *Procesamiento de Lenguaje Natural (NLP)*  

---

### *Título de la Tarea:*  
*Técnica de prompting: Program-Aided Language (PAL)*

 *Fecha de entrega:* 25/04/2025

*Presentado por:*  
- Luis Fernando Mendoza Cardona

*Profesor:*  
Edwin Puertas

---

# Ejemplo de PAL usando mai-ds-r1:free vía API gratuita

Este notebook muestra cómo aplicar la técnica PAL (Program-Aided Language) utilizando un modelo de IA (mai-ds-r1:free) a través de una API gratuita de [openrouter.ai](https://openrouter.ai/). El modelo generará un código Python para resolver un problema, ejecutaremos ese código y mostraremos la respuesta.

## 1. Instalación de dependencias

In [ ]:
# !pip install requests

## 2. Configuración de la API

Para usar la API, obtenemos una clave gratuita en [openrouter.ai](https://openrouter.ai/).

In [ ]:
# clave de API de OpenRouter
API_KEY = "<API_KEY>"

## 3. Definición del prompt PAL

Vamos a pedirle al modelo que resuelva los siguientes problemas usando la técnica PAL

El prompt PAL guía al modelo para que genere primero el código Python que resuelve el problema y luego la respuesta final.

### Prompt
Se le da dos ejemplos al modelo y finalmente se le hace una pregunta que debe responder con base en los ejemplos

la estructura del prompt es:

- Un enunciado (Q).
- Una respuesta explicativa paso a paso (A).
- Finalizando con la pregunta que se espera que el modelo responda.

Cada paso en la respuesta está compuesta por texto explicativo y una línea de código Python ejecutable.


In [ ]:
pal_prompt = """
Q: Roger tiene 5 pelotas de tenis. Compra 2 latas más de pelotas. Cada lata tiene 3 pelotas. ¿Cuántas pelotas tiene ahora?
A: Roger empezó con 5 pelotas.
tennis_balls = 5
Compra 2 latas de 3 pelotas cada una
bought_balls = 2 * 3
La respuesta es:
answer = tennis_balls + bought_balls

Q: Una panadería horneó 200 panes. Vendió 93 en la mañana y 39 en la tarde. Luego, un supermercado devolvió 6 panes. ¿Cuántos panes le quedan?
A: La panadería horneó 200 panes.
loaves_baked = 200
Vendieron 93 por la mañana y 39 por la tarde.
loaves_sold_morning = 93
loaves_sold_afternoon = 39
Un supermercado devolvió 6 panes.
loaves_returned = 6
La respuesta es:
answer = loaves_baked - loaves_sold_morning - loaves_sold_afternoon + loaves_returned

Q: Una tienda tenía 150 botellas de agua. Vendió 48 por la mañana y 36 por la tarde. Luego recibió un pedido de 20 botellas más. ¿Cuántas botellas tiene ahora?
A:
"""



In [ ]:
pal_prompt_1 = """
Q: A delivery service charges a flat fee of $5 plus $2 per mile traveled.
Write a function that calculates the total delivery cost for a given distance.
What is the cost for a 10-mile delivery?
A: First, we define a function that takes the number of miles as input.
def calculate_delivery_cost(miles):
    flat_fee = 5
    per_mile_rate = 2
    return flat_fee + per_mile_rate * miles

Now we call the function with 10 miles.
answer = calculate_delivery_cost(10)

Q: A factory has 4 machines. Each machine produces 12 parts per hour.
If the factory works for 6 hours, how many parts will be produced in total?
A: First, we define a function that takes the number of machines and hours as input.
def total_parts_produced(machines, hours):
    parts_per_hour = 12
    return machines * hours * parts_per_hour

Now we call the function with 4 machines and 6 hours.
answer = total_parts_produced(4, 6)

Q: A bookstore gives a 10% discount if the total purchase exceeds $100.
Write a function that takes a purchase amount and returns the final amount after applying the discount if applicable.
What is the final price for a $120 purchase?
A:
"""


In [ ]:
pal_prompt_2 = """
You are an assistant that solves word problems using Python code.
For each question, respond with:
1. Apply Chain of thought reasoning to explain your reasoning.
2. A well-structured Python function.
3. A function call that stores the result in a variable called `answer`.

Do not include explanations after the function call. End each response with the line: `answer = ...`

Q: A delivery service charges a flat fee of $5 plus $2 per mile traveled. Write a function that calculates the total delivery cost for a given distance. What is the cost for a 10-mile delivery?
A: First, we define a function that takes the number of miles as input.
def calculate_delivery_cost(miles):
    flat_fee = 5
    per_mile_rate = 2
    return flat_fee + per_mile_rate * miles

Now we call the function with 10 miles.
answer = calculate_delivery_cost(10)

Q: A factory has 4 machines. Each machine produces 12 parts per hour. If the factory works for 6 hours, how many parts will be produced in total?
A: First, we define a function that takes the number of machines and hours as input.
def total_parts_produced(machines, hours):
    parts_per_hour = 12
    return machines * hours * parts_per_hour

Now we call the function with 4 machines and 6 hours.
answer = total_parts_produced(4, 6)

Q: A bookstore gives a 10% discount if the total purchase exceeds $100. Write a function that takes a purchase amount and returns the final amount after applying the discount if applicable. What is the final price for a $120 purchase?
A:
"""


## 4. Llamada a la API de Microsoft llamado: mai-ds-r1:free

### Descripción del modelo - obtenida de openrouter
MAI-DS-R1 is a post-trained variant of DeepSeek-R1 developed by the Microsoft AI team to improve the model’s responsiveness on previously blocked topics while enhancing its safety profile. Built on top of DeepSeek-R1’s reasoning foundation, it integrates 110k examples from the Tulu-3 SFT dataset and 350k internally curated multilingual safety-alignment samples. The model retains strong reasoning, coding, and problem-solving capabilities, while unblocking a wide range of prompts previously restricted in R1.

MAI-DS-R1 demonstrates improved performance on harm mitigation benchmarks and maintains competitive results across general reasoning tasks. It surpasses R1-1776 in satisfaction metrics for blocked queries and reduces leakage in harmful content categories. The model is based on a transformer MoE architecture and is suitable for general-purpose use cases, excluding high-stakes domains such as legal, medical, or autonomous systems.

In [ ]:
import requests

# url de openrouter para acceder al modelo
URL = "https://openrouter.ai/api/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

data = {
    "model": "microsoft/mai-ds-r1:free",  # modelo seleccionado para la tarea
    "messages": [
        {"role": "user", "content": pal_prompt}
    ],
    "max_tokens": 500,
    "temperature": 0
}

response = requests.post(url=URL, headers=headers, json=data)
result_1 = response.json()


## 5. Ejecución del código generado

A continuación, ejecutamos el código generado por el modelo para obtener la respuesta al problema.

In [ ]:
print(result_1['choices'][0]['message']['content'])

A: La tienda tenía 150 botellas de agua.
bottles_initial = 150
Vendió 48 por la mañana y 36 por la tarde.
bottles_sold_morning = 48
bottles_sold_afternoon = 36
Recibió un pedido de 20 botellas.
bottles_received = 20
La respuesta es:
answer = bottles_initial - bottles_sold_morning - bottles_sold_afternoon + bottles_received


In [ ]:
data_1 = {
    "model": "microsoft/mai-ds-r1:free",
    "messages": [
        {"role": "user", "content": pal_prompt_1}
    ],
    "max_tokens": 500,
    "temperature": 0
}
response = requests.post(url=URL, headers=headers, json=data_1)
result = response.json()

In [ ]:
print(result['choices'][0]['message']['content'])

A: First, we define a function that checks if the purchase amount qualifies for a discount.
def calculate_final_price(amount):
    if amount > 100:
        return amount * 0.9  # Apply 10% discount
    else:
        return amount

Now we call the function with $120.
answer = calculate_final_price(120)

The final price is $108.0.


In [ ]:
data_2 = {
    "model": "microsoft/mai-ds-r1:free",
    "messages": [
        {"role": "user", "content": pal_prompt_2}
    ],
    "max_tokens": 500,
    "temperature": 0
}
response = requests.post(url=URL, headers=headers, json=data_2)
result_2 = response.json()
print(result_2['choices'][0]['message']['content'])

First, we define a function that checks if the purchase amount exceeds $100 to apply a 10% discount.
def calculate_final_price(amount):
    if amount > 100:
        return amount * 0.9
    else:
        return amount

Now we call the function with $120.
answer = calculate_final_price(120)


### Comprobación de la respuesta dada por el modelo

In [ ]:
def calculate_final_price(amount):
    if amount > 100:
        return amount * 0.9
    else:
        return amount
answer = calculate_final_price(120)
print(f"The final price is: {answer}")

The final price is: 108.0
